In [1]:
import os
import impyute
from utils.read_file  import readNex,saveData,lableEncoder

from logger import logger
import time
import tqdm
from baseline.SOTABaselineKNN import  imputeMethodKNN as KNN
from baseline.SOTABaselineMICE import  imputeMethodMICE as MICE
from baseline.SOTABaselineRandom import  imputeMethodRandom as Random
from baseline.SOTABaselineMedian import  imputeMethodMedain as Medain
from baseline.SOTABaselineRF import imputeMethodMR as MR
from baseline.myMethodTest import imputeMethod as TAI

from utils.handle_missingdata import gene_missingdata, gene_missingdata_taxa_bias, gene_missingdata_chara_bias, \
    gene_missingdata_block_bias

Using TensorFlow backend.


In [3]:
#原始含缺失数据集路径
originDataPath=r'C:\Users\pro\Desktop\实验相关文档\缺失插补建树全流程\起始含缺失数据'
#随机插补后无缺失数据集路径
noMissingDataPath=r'C:\Users\pro\Desktop\实验相关文档\缺失插补建树全流程\随机插补无缺失数据'

In [5]:

#生成模拟数据集
for file in os.listdir(originDataPath):
    data,missRow,speciesName,begin,end=readNex(os.path.join(originDataPath,file))
    noMissingData = impyute.imputation.cs.random(data)
    saveData(noMissingDataPath,file,speciesName,noMissingData,begin,end)

In [4]:
imputedDataPath=r'C:\Users\pro\Desktop\实验相关文档\缺失插补建树全流程\缺失插补结果'
for file in tqdm.tqdm(os.listdir(noMissingDataPath)):
    originData,missRow,speciesName,begin,end= readNex(os.path.join(noMissingDataPath,file))
    for missPattern in ['normal']:
        result = {}
        for missRate in [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]:
            if missPattern == 'normal':
                missData = gene_missingdata(rate=missRate, data=originData)
            elif missPattern == 'taxa':
                missData = gene_missingdata_taxa_bias(rate=missRate, data=originData)
            elif missPattern == 'chara':
                missData = gene_missingdata_chara_bias(rate=missRate, data=originData)
            elif missPattern == 'block':
                missData = gene_missingdata_block_bias(rate=missRate, data=originData)
            else:
                raise Exception("缺失模式错误，请在'normal','taxa','chara','block'中选择对应模式")
            saveData(noMissingDataPath,"{}_{}".format(file,str(missRate)),speciesName,missData,begin,end)
            missData=lableEncoder(missData)
            result, RandomImputedData = Random(result, originData, missData, missRate, missPattern, 'disperse')
            saveData(os.path.join(imputedDataPath,"Random"),"{}_{}".format(file,str(missRate)),speciesName,RandomImputedData,begin,end)
            
            result, KNNImputedData = KNN(result, originData, missData, missRate, missPattern, 'disperse')
            saveData(os.path.join(imputedDataPath,"KNN"),"{}_{}".format(file,str(missRate)),speciesName,KNNImputedData,begin,end)
            
            result,MedainImputedData = Medain(result, originData, missData, missRate, missPattern)
            saveData(os.path.join(imputedDataPath,"Medain"),"{}_{}".format(file,str(missRate)),speciesName,MedainImputedData,begin,end)
            
            
            start = time.time()
            result, MICEImputedData = MICE(result, originData, missData, missRate, missPattern, 'disperse')
            firstImputedData = MICEImputedData
            for loss in ['MSELoss']:
                for autoMethod in ['Autoencoder']:
                    
                    result,TAIimputedData= TAI(result=result, firstImputedMethod=firstImputedMethod,
                                 firstImputedData=firstImputedData,
                                 loss=loss, autoMethod=autoMethod,
                                 originData=originData, missData=missData,
                                 missRate=missRate, missPattern=missPattern)
                    logger.info(
                        "改后{}-{}-{}训练耗时:{}".format(firstImputedMethod, loss, autoMethod, time.time() - start))
                    saveData(os.path.join(imputedDataPath,"ours"),"{}_{}".format(file,str(missRate)),speciesName,TAIimputedData,begin,end)
    saveJson(result,
             os.path.join(imputedDataPath,"{}_{}_{}_{}.json".format("allmethod", missPattern, file,
                                       datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))))

  0%|                                                                                                                                                     | 0/8 [00:00<?, ?it/s]2021-01-13 18:58:58.335 | INFO     | utils.wapper:wapper:15 - imputeMethodRandom：耗时0.040001869201660156
2021-01-13 18:58:58.386 | INFO     | utils.wapper:wapper:15 - imputeMethodKNN：耗时0.044130563735961914
2021-01-13 18:58:58.398 | INFO     | utils.wapper:wapper:15 - imputeMethodMedain：耗时0.0060007572174072266


Imputing row 1/76 with 5 missing, elapsed time: 0.004


2021-01-13 18:59:18.279 | INFO     | utils.wapper:wapper:15 - imputeMethodMICE：耗时19.874874591827393
  0%|                                                                                                                                                     | 0/8 [00:20<?, ?it/s]


NameError: name 'firstImputedMethod' is not defined